# Process Aligned Registered images to identify the construction progress

In [ ]:
import os
from groundedvision.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
from groundedvision.projective_view import projective_view
from tqdm import tqdm

from loguru import logger

In [ ]:
path = f'{PROCESSED_DATA_DIR}/matched_pairs_json'
list_of_aligned_directories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]


### Generate the projective views of the 360 images 

In [ ]:
from pathlib import Path

for folder_path in list_of_aligned_directories:
    input_folder_path = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}"
    for image_path in os.listdir(input_folder_path):
        output_path = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}"
        full_path = os.path.join(input_folder_path, image_path)
        # check if the image patch contains
        if "_aligned.JPG" in full_path:
            projective_view(full_path, 120, output_path)

## Generate the Construction progress prompts

In [ ]:
import json
import re
from typing import List, Dict, Any
from openai import OpenAI

# ==========================================
# 1. DATA: Your Construction Taxonomy
# ==========================================
MARKDOWN_TABLE = """
| Class            | Category                                             | Stages                                 |
|:-----------------|:-----------------------------------------------------|:---------------------------------------|
| Wood             | Deck sheet - Wooden                                  | Deck sheet - Wooden                    |
| Wood             | Structural Beam- Wooden                              | Beam Installed                         |
| Concrete         | Footings                                             | Footings Installed                     |
| Concrete         | Micropiles                                           | Micropiles Installed                   |
| Concrete         | Soldier Piles                                        | Soldier Piles Installed                |
| Concrete         | Structural Columns - Concrete                        | Column Installed                       |
| Concrete         | Structural Slabs                                     | Slab Installed                         |
| Concrete         | Grade Slab                                           | Slab Installed                         |
| Concrete         | Slab Demolition                                      | Slab Demolished                        |
| Concrete         | Anchor Bolts                                         | Anchor Bolts Installed                 |
| Concrete         | Structural Beams - Concrete                          | Beams Installed                        |
| Concrete         | Plinth Beam                                          | Beams Installed                        |
| Concrete         | Double Tee                                           | Double Tee Installed                   |
| Concrete         | Fire Proofing                                        | Beams Fire Proofed                     |
| Concrete         | Fire Proofing                                        | Columns Fire Proofed                   |
| Concrete         | Screenwall                                           | Screen wall Installed                  |
| Concrete         | Vapour barrier                                       | Vapour barrier Installed               |
| Concrete         | parapet wall                                         | parapet wall Installed                 |
| Concrete         | Cat Walk Decking                                     | Deck Installed                         |
| Concrete         | Matt foundation                                      | Foundation installed                   |
| Concrete         | Trenching                                            | Trench installed                       |
| Concrete         | CMU Walls (Concrete Masonry Unit)                    | CMU Wall Installed                     |
| Concrete         | Masonry                                              | Wall Installed                         |
| Concrete         | Structural Wall                                      | Walls Installed                        |
| Concrete         | Cast In Place                                        | Cast In Place                          |
| Concrete         | Concrete Wall                                        | Concrete Wall Installed                |
| Doors            | Doors                                                | Door Frame                             |
| Doors            | Doors                                                | Door Installed                         |
| Doors            | Doors                                                | Hardware                               |
| Doors            | Coiling doors                                        | Doors Installed                        |
| Doors            | Windows                                              | Windows Installed                      |
| Doors            | Cabinets                                             | Cabinet Installed                      |
| Doors            | Doors                                                | Frames - Hollow Metal                  |
| Doors            | Doors                                                | Swing Doors - HM                       |
| Doors            | Doors                                                | Access Doors                           |
| Walls            | Drywall + Framing                                    | Top Track                              |
| Walls            | Drywall + Framing                                    | Bottom Track                           |
| Walls            | Drywall + Framing                                    | Framing                                |
| Walls            | Drywall + Framing                                    | Insulation                             |
| Walls            | Drywall + Framing                                    | Top out                                |
| Walls            | Drywall + Framing                                    | Drywall Hung                           |
| Walls            | Drywall + Framing                                    | Tape & Float                           |
| Walls            | Drywall + Framing                                    | First Coat Paint                       |
| Walls            | Wooden Partitions                                    | Wooden Framing                         |
| Walls            | Wooden Partitions                                    | Plywood Installed                      |
| Walls            | Glass Partitions                                     | Framing                                |
| Walls            | Glass Partitions                                     | Glass Installed                        |
| Walls            | Dryfall paint                                        | Final Paint                            |
| Walls            | Wall Demolition                                      | Demolished Walls                       |
| Walls            | Light Weight Concrete Wall                           | Wall Installed                         |
| Walls            | Wall Tiling                                          | Wall Tiling installed                  |
| Walls            | Wall Cladding                                        | Wall Cladding installed                |
| Walls            | Green glass                                          | Green glass installed                  |
| Walls            | Backing                                              | Backing Installed                      |
| Walls            | Blocking                                             | Blocking Installed                     |
| Walls            | Metal backing(Drywall Blocking-Wood &Metal)          | Metal backing installed                |
| Flooring         | Flooring                                             | Epoxy Flooring                         |
| Flooring         | Flooring                                             | Vinyl Flooring                         |
| Flooring         | Flooring                                             | Carpet flooring                        |
| Flooring         | Flooring                                             | Concrete Flooring                      |
| Flooring         | Flooring                                             | Porcelain florring                     |
| Flooring         | Flooring                                             | Vinyl Composite Tiling                 |
| Flooring         | Flooring                                             | Welded sheet Vinyl Flooring            |
| Flooring         | Flooring                                             | Epoxy Resign Flooring                  |
| Flooring         | Flooring                                             | Resilient heterogeneous sheet flooring |
| Flooring         | Flooring                                             | Poly Aspartic flooring                 |
| Flooring         | Flooring                                             | Rubber Flooring                        |
| Flooring         | Flooring                                             | Dal tile flooring                      |
| Flooring         | Flooring                                             | Flooring Installed                     |
| Flooring         | Pavers                                               | Pavers Installed                       |
| Structural Steel | Structural Beams - Steel                             | Beams Installed                        |
| Structural Steel | Structural columns - Steel                           | Columns Installed                      |
| Structural Steel | Structural Joists                                    | Joists Installed                       |
| Structural Steel | Structural Ceiling Framing                           | Structural Framing                     |
| Structural Steel | Secondary steel Framing                              | Framing Installed                      |
| Structural Steel | Catwalk                                              | Catwalk Installed                      |
| Structural Steel | Deck slab                                            | Deck sheet Erected                     |
| Structural Steel | Ductile Iron Pile                                    | DIPs Installed                         |
| Structural Steel | Ductile Iron Pile                                    | DIPs Capped                            |
| Ceiling          | GWB - Ceiling                                        | Ceiling Grid                           |
| Ceiling          | GWB - Ceiling                                        | Drywall Hung                           |
| Ceiling          | ACT - Acoustic Ceiling Panels                        | Ceiling Grid                           |
| Ceiling          | ACT - Acoustic Ceiling Panels                        | Ceiling Panels Installed               |
| Ceiling          | Wooden Ceiling                                       | WPC Metal Framing                      |
| Ceiling          | Wooden Ceiling                                       | Wooden Panels Installed                |
| Ceiling          | Wall Protection                                      | Wall Protection applied                |
| Ceiling          | Sofits                                               | Framing                                |
| Ceiling          | Sofits                                               | Drywall Hung                           |
| Fire Protection  | Fire Protection                                      | Horizontal Piping                      |
| Fire Protection  | Smoke detectors                                      | Smoke detectors                        |
| Fire Protection  | Sprinklers                                           | Sprinklers Installed                   |
| Fire Protection  | Sprinklers                                           | Sprinklers Head Installed              |
| Fire Protection  | Fire Alarm                                           | Fire Alarm Installed                   |
| Fire Protection  | Speakers                                             | Speakers Installed                     |
| Fire Protection  | FA Strobe                                            | FA Strobe Installed                    |
| Fire Protection  | FA horn & Strobe                                     | FA horn & Strobe Installed             |
| Plumbing         | Plumbing-Domestic                                    | Horizontal Piping                      |
| Plumbing         | Underground Storm Sewer                              | Storm Piping Installed                 |
| Plumbing         | Underground Storm Sewer                              | Manhole Installed                      |
| Plumbing         | Underground Storm Sewer                              | Trench drain Installed                 |
| Plumbing         | Plumbing-Drainage                                    | Horizontal Piping                      |
| Plumbing         | Fuel Piping                                          | Fueling piping Installed               |
| Plumbing         | Plumbing Risers                                      | Risers Installed                       |
| Plumbing         | PEX Lines                                            | PEX pipes Installed                    |
| Electrical       | Electrical Outlets                                   | Outlet Box Installed                   |
| Electrical       | Transformer                                          | Transformer Installed                  |
| Electrical       | Generator                                            | Generator Installed                    |
| Electrical       | Cable Bus Duct                                       | Cable Bus Duct Installed               |
| Electrical       | Switch gear                                          | Switch gear Installed                  |
| Electrical       | UPS                                                  | UPS Installed                          |
| Electrical       | Switches                                             | Switches Installed                     |
| Electrical       | Electrical Conduit                                   | Conduit Pipe Installed                 |
| Electrical       | Cable Tray                                           | Cable Tray Installed                   |
| Electrical       | Sleeves                                              | Sleeves Installed                      |
| Electrical       | Electrical Lights                                    | Lights Installed                       |
| Electrical       | Light Fixtures                                       | Light Fixtures                         |
| Electrical       | Junction Box                                         | Junction Box                           |
| Mechanical       | Mechanical Duct                                      | Duct Installed                         |
| Mechanical       | Mechanical Duct                                      | Duct Insulation                        |
| Mechanical       | Mechanical Piping                                    | Pipes Installed                        |
| Mechanical       | Mechanical Piping                                    | Pipes Insulation                       |
| Mechanical       | Mechanical Piping Risers                             | Risers Installed                       |
| Mechanical       | Conveyor                                             | Conveyor Framing                       |
| Mechanical       | Conveyor                                             | Conveyor Belt Installed                |
| Mechanical       | AHU (Air Handling Unit)                              | AHU Installed                          |
| Mechanical       | FCU (Fan Coil Unit)                                  | FCU Installed                          |
| Mechanical       | Dehumidifiers                                        | Dehumidifiers Installed                |
| Mechanical       | HVAC (Heating, Ventilation and Air Conditioning)     | HVAC Installed                         |
| Mechanical       | VESDA (Very Early Smoke Detection Apparatus)  Piping | VESDA Pipes Installed                  |
| Mechanical       | Diffuser                                             | Diffuser Installed                     |
| Mechanical       | Exhaust Grill                                        | Exhaust Grill                          |
| Mechanical       | Linear Diffuser                                      | Linear Diffuser                        |
| Mechanical       | Mechanical Piping-Refrigerator                       | Pipes Installed                        |
| Mechanical       | Mechanical Piping-Chiilled water                     | Pipes Installed                        |
| Mechanical       | VAV (Variable Air Volume)                            | VAV Installed                          |
| Mechanical       | Eye Ball Diffuser                                    | Eye Ball Diffuser                      |
| Communication    | Cable Tray-Electrical                                | Cable Tray Installed                   |
| Communication    | Cable Tray-Telecom                                   | Cable Tray Installed                   |
| Communication    | Wireless Access Point                                | Wireless Access Point                  |
| Communication    | Occupancy sensor                                     | Occupancy sensor Installed             |
| Communication    | Access Panel                                         | Access Panel Installed                 |
| Communication    | Motion Detector/Glass Break                          | Motion Detector/Glass Break            |
"""

In [ ]:
def parse_markdown_taxonomy(md_table: str) -> List[Dict[str, str]]:
    """
    Parses a markdown table into a list of dictionaries with 
    keys: class_name, category, stage.
    """
    valid_triplets = []
    lines = md_table.strip().split('\n')
    
    # Skip header and separator lines (lines starting with | Class | and |:---)
    start_parsing = False
    
    for line in lines:
        if "Class" in line and "Category" in line:
            start_parsing = True
            continue
        if "---" in line:
            continue
        if not start_parsing:
            continue
            
        # Extract cells
        cells = [cell.strip() for cell in line.split('|') if cell.strip()]
        
        if len(cells) >= 3:
            triplet = {
                "class_name": cells[0],
                "category": cells[1],
                "stage": cells[2]
            }
            valid_triplets.append(triplet)
            
    return valid_triplets

In [ ]:
from groundedvision.pydantic_schema import (
    build_guided_schema_pydantic,       # Pydantic schema generation
)
triplets= parse_markdown_taxonomy(MARKDOWN_TABLE)
schema = build_guided_schema_pydantic(triplets)

In [ ]:
schema

In [ ]:
import os
import mimetypes
import base64

def encode_image(image_path: str) -> str:
    """
    Reads a local image file and converts it to a base64 data URI 
    compatible with the OpenAI/vLLM API.
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found at: {image_path}")
        
    # Guess the mime type (e.g., image/jpeg, image/png)
    mime_type, _ = mimetypes.guess_type(image_path)
    if not mime_type:
        mime_type = "image/jpeg" # Default fallback

    with open(image_path, "rb") as image_file:
        base64_encoded = base64.b64encode(image_file.read()).decode('utf-8')
        
    return f"data:{mime_type};base64,{base64_encoded}"

In [ ]:
def analyze_image(image_input: str, model_name: str = "Qwen/Qwen3-VL-Instruct"):
    # 1. Parse the taxonomy
    triplets = parse_markdown_taxonomy(MARKDOWN_TABLE)
    
    # 2. Build the strict schema
    schema = build_guided_schema(triplets)
    
    # 3. Prepare Image URL (Handle URL vs Local File)
    if image_input.startswith("http://") or image_input.startswith("https://"):
        final_image_url = image_input
    else:
        print(f"Encoding local image: {image_input}...")
        final_image_url = encode_image(image_input)

    # 4. Setup Client
    client = OpenAI(
        api_key="EMPTY", 
        base_url="http://localhost:8000/v1"
    )

    # 5. Define the Prompt
    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert Construction Progress Tracker. "
                "Your job is to identify construction elements in an image and "
                "classify them strictly according to the provided project taxonomy. "
                "You must output bounding boxes for every identified element."
            )
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text", 
                    "text": (
                        "Analyze this image. First, 'think' about the visual evidence (textures, materials). "
                        "Then, list every construction element found, selecting the EXACT Class, Category, and Stage "
                        "from the allowed list."
                    )
                },
                {
                    "type": "image_url",
                    "image_url": {"url": final_image_url}
                }
            ]
        }
    ]

    print("Sending request to Qwen3-VL...")
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=0.1,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "analysis_results",
                    "schema": schema
                },
            },
        )
        
        result_content = response.choices[0].message.content
        print(f"Response: {result_content}")
        parsed_result = json.loads(result_content)
        
        # Pretty Print Output
        print("\n=== Analysis Results ===")
        print(f"Thinking Process: {parsed_result.get('thought_process', 'N/A')}\n")
        print(f"{'CLASS':<15} | {'CATEGORY':<30} | {'STAGE':<25} | {'BBOX'}")
        print("-" * 90)
        
        for item in parsed_result.get("identified_objects", []):
            print(f"{item['class']:<15} | {item['category']:<30} | {item['stage']:<25} | {item['bbox_2d']}")
            
        return parsed_result

    except Exception as e:
        print(f"Error during inference: {e}")
        return None

In [ ]:
def analyze_images(image_input_old: str, image_input_new:str, model_name: str = "Qwen/Qwen3-VL-Instruct"):
    # 1. Parse the taxonomy
    triplets = parse_markdown_taxonomy(MARKDOWN_TABLE)

    print(f"Encoding local image: {image_input_old}...")
    final_image_url_old = encode_image(image_input_old)
    print(f"Encoding local image: {image_input_new}...")
    final_image_url_new = encode_image(image_input_new)

    # 4. Setup Client
    client = OpenAI(
        api_key="EMPTY", 
        base_url="http://localhost:8000/v1"
    )

    # 5. Define the Prompt with EXPLICIT output format instructions
    system_prompt = """You are an expert Construction Progress Tracker.
Your job is to identify construction elements in the images and classify them according to a project taxonomy.

You MUST respond with a JSON object using EXACTLY this structure:
{
  "thought_process": "<your step-by-step reasoning about what you see>",
  "identified_objects": [
    {
      "class": "<Class name from taxonomy>",
      "category": "<Category from taxonomy>",
      "stage": "<Stage from taxonomy>",
      "bbox_2d": [ymin, xmin, ymax, xmax]
    }
  ]
}

CRITICAL REQUIREMENTS:
1. Use EXACTLY these field names: "thought_process", "identified_objects", "class", "category", "stage", "bbox_2d"
2. bbox_2d must be an array of 4 integers: [ymin, xmin, ymax, xmax] in pixel coordinates
3. Select Class, Category, and Stage values ONLY from the provided taxonomy
4. Include ALL visible construction elements"""

    user_prompt = f"""Analyze this construction site image.

AVAILABLE TAXONOMY (Class | Category | Stage):
{MARKDOWN_TABLE}

Instructions:
1. First, observe what materials, textures, and construction elements do you see. This is to identify the difference between both the images
2. Then, list differences between both the images and return them in "identified_objects" with:
   - "class": The main class from the taxonomy (e.g., "Concrete", "Structural Steel", "Mechanical")
   - "category": The specific category (e.g., "Structural Beams - Steel", "Mechanical Duct")
   - "stage": The construction stage (e.g., "Beams Installed", "Duct Installed")
   - "bbox_2d": Bounding box as [ymin, xmin, ymax, xmax] integers

Remember: Use the EXACT field names specified. Do NOT use alternative names like "bounding_box" or "elements"."""

    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": user_prompt},
                {"type": "image_url", "image_url": {"url": final_image_url_old}},
                {"type": "image_url", "image_url": {"url": final_image_url_new}}
            ]
        }
    ]

    print("Sending request to Qwen3-VL...")
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=0.1,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "analysis_results",
                    "schema": schema
                },
            },
        )
        
        result_content = response.choices[0].message.content
        reasoning = response.choices[0].message.reasoning
        print(f"Response: {result_content}")
        print(f"Reasoning: {reasoning}")
        parsed_result = json.loads(result_content)
        # Handle both list and dict responses
        if isinstance(parsed_result, list):
            identified_objects = parsed_result
        else:
            identified_objects = parsed_result.get("identified_objects", [])
        # Pretty Print Output
        print("\n=== Analysis Results ===")
        print(f"{'CLASS':<15} | {'CATEGORY':<30} | {'STAGE':<25} | {'BBOX'}")
        print("-" * 90)
        for item in identified_objects:
            print(f"{item['class']:<15} | {item['category']:<30} | {item['stage']:<25} | {item['bbox_2d']}")
                    
        return parsed_result

    except Exception as e:
        print(f"Error during inference: {e}")
        return None

In [ ]:
# Specify your local file name here
LOCAL_IMAGE_PATH_OLD_FOLDER = "/media/satish/DATA/Development/workspace/projects/GroundedVision/data/processed/matched_pairs_json/97_6d6f8b18-4a8f-4011-a540-a29f266ffcd8/old_VID_20251210_123543_00_325_IMG_00491_aligned_cube_map/" 
LOCAL_IMAGE_PATH_NEW_FOLDER = "/media/satish/DATA/Development/workspace/projects/GroundedVision/data/processed/matched_pairs_json/97_6d6f8b18-4a8f-4011-a540-a29f266ffcd8/new_VID_20251217_151743_00_132_IMG_00110_aligned_cube_map/" 


In [ ]:
from PIL import Image
from pathlib import Path

img_path_old = Path(LOCAL_IMAGE_PATH_OLD)
img_path_new = Path(LOCAL_IMAGE_PATH_NEW)
img_old = Image.open(img_path_old)
img_new = Image.open(img_path_new)

In [ ]:
display(img_new)

In [ ]:

# Rotate 90 degrees clockwise
rotated_img_old = img_old.transpose(Image.Transpose.ROTATE_270)
# Build new filename: original_name_rotated.extension
rotated_image_path_old = img_path_old.parent / f"{img_path_old.stem}_rotated{img_path_old.suffix}"
# Save the rotated image
rotated_img_old.save(rotated_image_path_old)


# Rotate 90 degrees clockwise
rotated_img_new = img_new.transpose(Image.Transpose.ROTATE_270)
# Build new filename: original_name_rotated.extension
rotated_image_path_new = img_path_new.parent / f"{img_path_new.stem}_rotated{img_path_new.suffix}"
# Save the rotated image
rotated_img_new.save(rotated_image_path_new)

In [ ]:
display(rotated_img_old)

In [ ]:
display(rotated_img_new)

In [ ]:
# Ensure this matches your running vLLM model name
MODEL_SLUG = "Qwen/Qwen3-VL-8B-Thinking-FP8" 

In [ ]:
analyze_images(rotated_image_path_old, rotated_image_path_new, model_name=MODEL_SLUG)


In [ ]:
from groundedvision.construction import (
    analyze_image, 
    analyze_mep, 
    analyze_all_domains,
    MEP_CLASSES
)
# Full taxonomy analysis
result = analyze_image(str(new_path), model_name=MODEL_SLUG)
# MEP only (smaller context, faster, more focused)
result = analyze_mep(str(new_path), model_name=MODEL_SLUG)
# Custom classes
result = analyze_image(str(new_path), model_name=MODEL_SLUG, classes=["Electrical", "Plumbing"])
# Run all domains sequentially
results = analyze_all_domains(str(new_path), model_name=MODEL_SLUG)

In [ ]:
from groundedvision.construction import (
    analyze_image, 
    analyze_mep, 
    analyze_all_domains,
    MEP_CLASSES
)
# Custom classes
result = analyze_image(str(rotated_image_path_new), model_name=MODEL_SLUG, classes=["Electrical", "Plumbing"])

In [ ]:

# Custom classes
result = analyze_image(str(rotated_image_path_old), model_name=MODEL_SLUG, classes=["Electrical", "Plumbing"])

In [ ]:
import json
import random
import io
import ast
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
from PIL import ImageColor

additional_colors = [colorname for (colorname, colorcode) in ImageColor.colormap.items()]

def plot_bounding_boxes(im, bounding_boxes):
    """
    Plots bounding boxes on an image with markers for each a name, using PIL, normalized coordinates, and different colors.

    Args:
        img_path: The path to the image file.
        bounding_boxes: A list of bounding boxes containing the name of the object
         and their positions in normalized [y1 x1 y2 x2] format.
    """

    # Load the image
    img = im
    width, height = img.size
    print(img.size)
    # Create a drawing object
    draw = ImageDraw.Draw(img)

    # Define a list of colors
    colors = [
    'red',
    'green',
    'blue',
    'yellow',
    'orange',
    'pink',
    'purple',
    'brown',
    'gray',
    'beige',
    'turquoise',
    'cyan',
    'magenta',
    'lime',
    'navy',
    'maroon',
    'teal',
    'olive',
    'coral',
    'lavender',
    'violet',
    'gold',
    'silver',
    ] + additional_colors

    font = ImageFont.truetype("NotoSansCJK-Regular.ttc", size=14)

    # Iterate over the bounding boxes
    for i, bounding_box in enumerate(bounding_boxes):
      # Select a color from the list
      color = colors[i % len(colors)]

      # Convert normalized coordinates to absolute coordinates
      abs_y1 = int(bounding_box["bbox_2d"][1] / 1000 * height)
      abs_x1 = int(bounding_box["bbox_2d"][0] / 1000 * width)
      abs_y2 = int(bounding_box["bbox_2d"][3] / 1000 * height)
      abs_x2 = int(bounding_box["bbox_2d"][2] / 1000 * width)

      if abs_x1 > abs_x2:
        abs_x1, abs_x2 = abs_x2, abs_x1

      if abs_y1 > abs_y2:
        abs_y1, abs_y2 = abs_y2, abs_y1

      # Draw the bounding box
      draw.rectangle(
          ((abs_x1, abs_y1), (abs_x2, abs_y2)), outline=color, width=3
      )

      # Draw the text
      if "label" in bounding_box:
        draw.text((abs_x1 + 8, abs_y1 + 6), bounding_box["label"], fill=color, font=font)

    # Display the image
    return img


In [ ]:
bounding_boxes = []
for dict_val in result['identified_objects']:
    transformed_dict_val = {
        "label": dict_val['class']+ " " + dict_val['category'] + " " + dict_val['stage'],
        "bbox_2d": dict_val['bbox_2d']
    }
    bounding_boxes.append(transformed_dict_val)


In [ ]:
plot_bounding_boxes(rotated_img, bounding_boxes)

In [ ]:

    
# Run
if os.path.exists(LOCAL_IMAGE_PATH):
    analyze_image(LOCAL_IMAGE_PATH, MODEL_SLUG)
else:
    print(f"Please place an image named '{LOCAL_IMAGE_PATH}' in this directory or update the path.")

In [ ]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

models = client.models.list()
model = models.data[0].id

# Round 1
messages = [{"role": "user", "content": "9.11 and 9.8, which is greater?"}]
# For granite, add: `extra_body={"chat_template_kwargs": {"thinking": True}}`
# For Qwen3 series, if you want to disable thinking in reasoning mode, add:
# extra_body={"chat_template_kwargs": {"enable_thinking": False}}
response = client.chat.completions.create(model=model, messages=messages,
    extra_body={"chat_template_kwargs": {"enable_thinking": True}})
reasoning = response.choices[0].message.reasoning
content = response.choices[0].message.content


In [ ]:

print("reasoning:", reasoning)

In [ ]:

print("content:", content)